In [1]:
import pandas as pd
import openai
from dotenv import load_dotenv
import os
import json
from tqdm import tqdm
from pathlib import Path

load_dotenv(dotenv_path=".env")

True

In [2]:
openai.api_key = os.getenv("OPENAI_API_KEY")
print("OPENAI API KEY LOADED:", bool(openai.api_key))

client = openai.OpenAI(api_key=openai.api_key)
print("OpenAI CLIENT INITIALIZED:", bool(client))

OPENAI API KEY LOADED: True
OpenAI CLIENT INITIALIZED: True


In [3]:
def load_prompt(prompt_path):
    prompt_path = Path(prompt_path)
    with open(prompt_path, "r", encoding="utf-8") as f:
        return f.read()


In [4]:
def analyze_review(review_text, star_rating, prompt_template):
    # Replace placeholder with actual review
    prompt = (prompt_template
              .replace("{review_text}", review_text or "")
              .replace("{star_rating}", str(star_rating)))

    try:
        response = client.chat.completions.create(
            model="gpt-5-nano",
            messages=[{"role": "user", "content": prompt}]
        )

        content = response.choices[0].message.content.strip()
        return json.loads(content)

    except json.JSONDecodeError:
        return {
            "error": "Invalid JSON returned by model",
            "raw_output": content if 'content' in locals() else None
        }
    except Exception as e:
        return {"error": str(e)}


In [5]:
sample_review = "The book was interesting to read but hard to download."
star_rating = 5
prompt_path = "prompts/prompt2.txt"
result = analyze_review(sample_review, star_rating, load_prompt(prompt_path))
result

{'overall_sentiment': 'Neutral',
 'aspect_sentiments': {'Product Quality': 'N/A',
  'Price & Value': 'N/A',
  'Usability': 'N/A',
  'Access & Compatibility': 'Negative',
  'Customer Support': 'N/A',
  'Content Variety & Features': 'Positive',
  'Other': 'N/A'},
 'main_problem': 'Access issue (download)',
 'reason': 'Difficulty downloading the book.',
 'text_rating': 3}

In [6]:
def process_reviews(
    df,
    prompt_path,
    output_path,
    max_samples=None,
    save_every=4000  # autosave every 4000 rows
):

    PROMPT_TEMPLATE = load_prompt(prompt_path)

    # limit to first n rows if needed
    if max_samples:
        df = df.head(max_samples)
    df = df.reset_index(drop=True)

    # Resume if previous file exists
    if os.path.exists(output_path):
        print(f"Resuming from {output_path}...")
        processed_df = pd.read_csv(output_path)
        start_idx = len(processed_df)
        print(f"Found {start_idx} already processed rows, resuming...")
    else:
        processed_df = pd.DataFrame(columns=list(df.columns) + ["analysis_result"])
        start_idx = 0

    # Processing loop
    for i in tqdm(range(start_idx, len(df)), total=len(df) - start_idx, desc="Processing reviews"):
        row = df.iloc[i]
        review_text = row.get("review_body", "")
        star_rating = row.get("star_rating", 0)

        try:
            result = analyze_review(review_text, star_rating, PROMPT_TEMPLATE)
        except Exception as e:
            result = {"error": str(e)}

        # Add row with JSON string
        processed_df.loc[len(processed_df)] = list(row.values) + [str(result)]

        # Save every batch or at the very end
        if (i + 1) % save_every == 0 or (i + 1) == len(df):
            processed_df.to_csv(output_path, index=False)
            print(f"Progress saved at {i + 1} rows")

    return processed_df


In [7]:
df = pd.read_csv('merged_digital_2012.csv.zip')
df.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,year,month,review_length,length_bin
0,US,52496677,R1RHTGGZMYAH2Y,B002VQ23AG,400330899,"Brahms, J. / Strauss, R. / Herzogenberg, H.: C...",Digital_Music_Purchase,5,1,1,False,False,Johannes Moser Continues his survey of Brahms ...,Johannes Moser continues his successful foray ...,2012-02-17,2012.0,2012-02,362,long
1,US,46707532,R23WEYK28ZEF4P,B0050VI69G,70822140,Pumped Up Kicks,Digital_Music_Purchase,4,0,0,False,False,"Great tune, weird lyrics",Pumped up kicks has a really nice tune. Shure ...,2012-11-04,2012.0,2012-11,68,medium
2,US,53068061,R17FAB2JSQHIGL,B005GRUOHG,737519878,Detoxing,Digital_Music_Purchase,3,0,0,False,False,Detoxing - this is a good thing !,Norway has always had it's share of highly ski...,2012-02-29,2012.0,2012-02,295,long
3,US,13232448,R2IQ8G3C02ZIWM,B004FCEFUK,275286673,The Great Escape,Digital_Music_Purchase,4,3,4,False,True,Amazing album marred by a few cheesy tracks,Seventh Wonder is a progressive power metal ba...,2012-12-18,2012.0,2012-12,521,extra_long
4,US,36129886,R3GHM0W5QL5ZF9,B009DP9D6A,546585055,> album title goes here < [Explicit],Digital_Music_Purchase,4,3,4,False,False,"How do you like your ""coffee""...??",The album is not as poor as some of the review...,2012-09-30,2012.0,2012-09,895,extra_long


In [12]:
prompt_path = "prompts/prompt2.txt"
output_path = "analyzed_review_data/prompt2_processed_reviews.csv"
r = process_reviews(df, prompt_path, output_path, max_samples=100)
r

Resuming from analyzed_review_data/prompt2_processed_reviews.csv...
Found 4 already processed rows, resuming...


Processing reviews: 100%|██████████| 96/96 [13:29<00:00,  8.44s/it]

Progress saved at 100 rows


,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,year,month,review_length,length_bin,analysis_result
0,US,52496677,R1RHTGGZMYAH2Y,B002VQ23AG,400330899,"Brahms, J. / Strauss, R. / Herzogenberg, H.: C...",Digital_Music_Purchase,5,1,1,False,False,Johannes Moser Continues his survey of Brahms ...,Johannes Moser continues his successful foray ...,2012-02-17,2012.0,2012-02,362,long,"{'overall_sentiment': 'Positive', 'aspect_sent..."
1,US,46707532,R23WEYK28ZEF4P,B0050VI69G,70822140,Pumped Up Kicks,Digital_Music_Purchase,4,0,0,False,False,"Great tune, weird lyrics",Pumped up kicks has a really nice tune. Shure ...,2012-11-04,2012.0,2012-11,68,medium,"{'overall_sentiment': 'Positive', 'aspect_sent..."
2,US,53068061,R17FAB2JSQHIGL,B005GRUOHG,737519878,Detoxing,Digital_Music_Purchase,3,0,0,False,False,Detoxing - this is a good thing !,Norway has always had it's share of highly ski...,2012-02-29,2012.0,2012-02,295,long,"{'overall_sentiment': 'Positive', 'aspect_sent..."
3,US,13232448,R2IQ8G3C02ZIWM,B004FCEFUK,275286673,The Great Escape,Digital_Music_Purchase,4,3,4,False,True,Amazing album marred by a few cheesy tracks,Seventh Wonder is a progressive power metal ba...,2012-12-18,2012.0,2012-12,521,extra_long,"{'overall_sentiment': 'Positive', 'aspect_sent..."
4,US,36129886,R3GHM0W5QL5ZF9,B009DP9D6A,546585055,> album title goes here < [Explicit],Digital_Music_Purchase,4,3,4,False,False,"How do you like your ""coffee""...??",The album is not as poor as some of the review...,2012-09-30,2012.0,2012-09,895,extra_long,"{'overall_sentiment': 'Positive', 'aspect_sent..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,US,51352061,RZ912IW2D1U4Z,B004J0NBUI,543269650,Depression Free - Binaural Beat Brainwave Subl...,Digital_Music_Purchase,3,29,34,False,False,Not good for severe sensory overload.,I have fibromyalgia and often suffer from sens...,2012-04-25,2012.0,2012-04,55,medium,"{'overall_sentiment': 'Negative', 'aspect_sent..."
96,US,52518703,R3LPMH73DQCXM9,B007J29OU4,389233204,Port Of Morrow,Digital_Music_Purchase,4,1,2,False,False,Dad rock but still addictive,"The term \""dad rock\"" has always annoyed me, w...",2012-04-09,2012.0,2012-04,996,extra_long,"{'overall_sentiment': 'Positive', 'aspect_sent..."
97,US,40230771,R27G6AIO4WKOFT,B008671DK0,858905738,Bear Creek,Digital_Music_Purchase,5,0,0,False,True,Enter with a roar!!!,The first thing that comes to mind about Brand...,2012-11-21,2012.0,2012-11,776,extra_long,"{'overall_sentiment': 'Positive', 'aspect_sent..."
98,US,53037408,RFWU0V6OFH3OH,B000UB335S,736836111,Thelonious Monk Trio [RVG Remaster],Digital_Music_Purchase,5,7,7,False,False,To RVG or not to RVG? That is the question.,"First, this is an excellent album of some earl...",2012-05-27,2012.0,2012-05,371,long,"{'overall_sentiment': 'Positive', 'aspect_sent..."
